In [8]:
import os
from pathlib import Path
from gurobipy import Model, GRB
from data_loader import load_prices, load_storage, load_efficiency, load_plant_capacity, load_demand
import pandas as pd

# Data Loading from data folder in the parent directory
current_dir = Path().resolve()
data_folder = os.path.join(current_dir.parent, "data")
fuel_prices_file = os.path.join(data_folder, "fuel_prices.csv")           # Price in EUR/tonne (coal,oil) or EUR/m3 (gas)
storage_file = os.path.join(data_folder, "storage.csv")         # Max storage in tonnes/m3
efficiency_file = os.path.join(data_folder, "efficiency.csv")   # efficiency of MWh th to MWh el
plant_file = os.path.join(data_folder, "plant_capacity.csv")    # Plant capacity in MWh/month
demand_file = os.path.join(data_folder, "electricity_demand.csv")           # Monthly demand in MWh

start_date = "2020-01"
end_date = "2020-12"

fuels = ["coal", "oil", "gas"]
zone = 'DK_2'

try:
    fuel_prices_df = load_prices(fuel_prices_file, start_date, end_date)
    storage = load_storage(storage_file)
    efficiency = load_efficiency(efficiency_file)
    plant_capacity = load_plant_capacity(plant_file)
    demand_df = load_demand(demand_file, start_date, end_date, zone=zone ,supply_factor=0.0005)
except Exception as e:
    raise RuntimeError(f"Error loading data: {e}")


# Timesteps
T = list(range(len(demand_df)))

# Model Initialization

m = Model("Fuel_Procurement_OneMonth")

F = [fuel for fuel in fuel_prices_df.columns if fuel in fuels]

# Decision Variables
# Decision Variables (corrected filters)
x = {(fuel, t): m.addVar(lb=0, ub=storage[fuel], name=f"buy_{fuel}_{t}")
     for fuel in F for t in T}

y = {(fuel, t): m.addVar(lb=0, ub=plant_capacity[fuel], name=f"gen_{fuel}_{t}")
     for fuel in F for t in T}

s = {(fuel, t): m.addVar(lb=0, ub=storage[fuel], name=f"s_{fuel}_{t}")
     for fuel in F for t in T}




m.update()



In [ ]:
# Initial Storage Levels
initial_storage = {fuel: 0 for fuel in fuel_prices_df.columns if fuel in fuels}

# Constraints

# --- Add consumption (withdrawal) variables so stored fuel can be used later ---
# w = fuel consumed in a period (units same as storage units MWh th)
w = {(fuel, t): m.addVar(lb=0, ub=storage[fuel], name=f"cons_{fuel}_{t}")
     for fuel in F for t in T}

m.update()

# Demand constraint: one per time period
for t in T:
    m.addConstr(
        sum(y[fuel, t] for fuel in F) >= int(demand_df.iloc[t][zone]),
        name=f"Demand_Constraint_{t}"
    )

# efficiency constraints
for fuel in F:
    for t in T:
        m.addConstr(
            y[fuel, t] == efficiency[fuel] * w[fuel, t],
            name=f"Gen_from_Consumption_{fuel}_{t}"
        )

m.update()

# State-of-energy (SOE) constraints: storage evolves with buys (x) and withdrawals (w)
for fuel in F:
    print(f"Setting up SOE constraints for {fuel}")
    for t in T:
        print(f"  Time period {t}")
        if t == 0:
            m.addConstr(
                s[fuel, t] == initial_storage[fuel] + x[fuel, t] - w[fuel, t],
                name=f"SOE_Def_{fuel}_{t}"
            )
        else:
            m.addConstr(
                s[fuel, t] == s[fuel, t-1] + x[fuel, t] - w[fuel, t],
                name=f"SOE_Def_{fuel}_{t}"
            )

# Storage limits (explicit upper bound)
for fuel in F:
    for t in T:
        m.addConstr(
            s[fuel, t] <= storage[fuel],
            name=f"SOE_Limit_{fuel}_{t}"
        )

# Cyclic SOE (start == end)
t0 = 0
tlast = T[-1]   # last period
for fuel in F:
    m.addConstr(
        s[fuel, t0] == s[fuel, tlast],
        name=f"SOE_Cycle_{fuel}"
    )

# Objective
m.setObjective(
    sum(fuel_prices_df.iloc[t][fuel] * x[fuel, t] for fuel in F for t in T),
    GRB.MINIMIZE
)
m.update()

# Optimize
m.optimize()

# print results
if m.status == GRB.OPTIMAL:
    print("Optimal Solution Found:")
    # Purchased and generated per fuel and period
    for (fuel, t) in sorted(x.keys()):
        print(f"Purchased {x[fuel, t].X:.4f} units of {fuel} at period {t}")
    for (fuel, t) in sorted(y.keys()):
        print(f"Generated {y[fuel, t].X:.4f} MWh from {fuel} at period {t}")
    for (fuel, t) in sorted(s.keys()):
        print(f"Stored {s[fuel, t].X:.4f} units of {fuel} at period {t}")

    # Cost per fuel aggregated across periods
    total_cost = 0.0
    for fuel in F:
        cost_f = sum(fuel_prices_df.iloc[t][fuel] * x[fuel, t].X for t in T)
        print(f"Cost for {fuel}: EUR {cost_f:.2f}")
        total_cost += cost_f

    print(f"Total Cost: EUR {total_cost:.2f}")
else:
    print("No optimal solution found.")


Setting up SOE constraints for coal
  Time period 0
  Time period 1
  Time period 2
  Time period 3
  Time period 4
  Time period 5
  Time period 6
  Time period 7
  Time period 8
  Time period 9
  Time period 10
  Time period 11
Setting up SOE constraints for oil
  Time period 0
  Time period 1
  Time period 2
  Time period 3
  Time period 4
  Time period 5
  Time period 6
  Time period 7
  Time period 8
  Time period 9
  Time period 10
  Time period 11
Setting up SOE constraints for gas
  Time period 0
  Time period 1
  Time period 2
  Time period 3
  Time period 4
  Time period 5
  Time period 6
  Time period 7
  Time period 8
  Time period 9
  Time period 10
  Time period 11
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 25.1.0 25B78)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 123 rows, 144 columns and 291 nonzeros
Model fingerprint: 0x11aa3134
Coefficient statistics:
  Matrix ran

In [ ]:
# ---- Create results dataframe ----
month_end_dates = pd.date_range(start=start_date, end="2020-12-31", freq="M")
rows = []
acc_cost = 0.0

for idx, t in enumerate(T):
    row = {"Month": month_end_dates[idx].strftime("%Y-%m-%d")}
    cost_t = 0.0

    for fuel in F:
        bought = x[fuel, t].X
        generated = y[fuel, t].X

        row[f"{fuel}_Bought"] = bought
        row[f"{fuel}_Generated"] = generated

        cost_t += fuel_prices_df.iloc[t][fuel] * bought

    acc_cost += cost_t

    row["Cost_EUR"] = cost_t
    row["Accumulated_Cost"] = acc_cost

    rows.append(row)

results_df = pd.DataFrame(rows)

# Rename to capitalized column names
rename_map = {
    "coal_Bought": "Coal_Bought",
    "oil_Bought": "Oil_Bought",
    "gas_Bought": "Gas_Bought",
    "coal_Generated": "Coal_Generated",
    "oil_Generated": "Oil_Generated",
    "gas_Generated": "Gas_Generated",
    "coal_Stored": "Coal_Stored",
    "oil_Stored": "Oil_Stored",
    "gas_Stored": "Gas_Stored",
    
}
results_df = results_df.rename(columns=rename_map)

# Reorder columns
desired_order = [
    "Month",
    "Cost_EUR",
    "Accumulated_Cost",
    "Coal_Bought",
    "Oil_Bought",
    "Gas_Bought",
    "Coal_Generated",
    "Oil_Generated",
    "Gas_Generated",
    "Coal_Stored", 
    "Oil_Stored", 
    "Gas_Stored"
]
results_df = results_df[desired_order]



# ---- Save CSV to folder ----
output_path = "/Users/rofrug/Library/CloudStorage/OneDrive-DanmarksTekniskeUniversitet/First Year PhD/46750 Optimization in modern power systems/46750_g28_Assignment_2/results/model2"
os.makedirs(output_path, exist_ok=True)

csv_file = os.path.join(output_path, "running_cost_results.csv")
results_df.to_csv(csv_file, index=False)

print(f"Results saved to: {csv_file}")

Results saved to: /Users/rofrug/Library/CloudStorage/OneDrive-DanmarksTekniskeUniversitet/First Year PhD/46750 Optimization in modern power systems/46750_g28_Assignment_2/results/model2/running_cost_results.csv


/var/folders/8b/6z2b3l6952gg8zq23ybm69hr0000z9/T/ipykernel_13578/2082229290.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_end_dates = pd.date_range(start=start_date, end="2020-12-31", freq="M")
